# Configure Gmail API

In [2]:
import pickle
import os
from google_auth_oauthlib.flow import Flow, InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.http import MediaFileUpload, MediaIoBaseDownload
from google.auth.transport.requests import Request



def Create_Service(client_secret_file, api_name, api_version, *scopes):
    print(client_secret_file, api_name, api_version, scopes, sep='-')
    CLIENT_SECRET_FILE = client_secret_file
    API_SERVICE_NAME = api_name
    API_VERSION = api_version
    SCOPES = [scope for scope in scopes[0]]
    print(SCOPES)

    cred = None

    pickle_file = f'token_{API_SERVICE_NAME}_{API_VERSION}.pickle'
    # print(pickle_file)

    if os.path.exists(pickle_file):
        with open(pickle_file, 'rb') as token:
            cred = pickle.load(token)

    if not cred or not cred.valid:
        if cred and cred.expired and cred.refresh_token:
            cred.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(CLIENT_SECRET_FILE, SCOPES)
            cred = flow.run_local_server()

        with open(pickle_file, 'wb') as token:
            pickle.dump(cred, token)

    try:
        service = build(API_SERVICE_NAME, API_VERSION, credentials=cred)
        print(API_SERVICE_NAME, 'service created successfully')
        return service
    except Exception as e:
        print('Unable to connect.')
        print(e)
        return None

def convert_to_RFC_datetime(year=1900, month=1, day=1, hour=0, minute=0):
    dt = datetime.datetime(year, month, day, hour, minute, 0).isoformat() + 'Z'
    return dt

  
import base64
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

CLIENT_SECRET_FILE = 'client_secrets.json'
API_NAME = 'gmail'
API_VERSION = 'v1'
SCOPES = ['https://mail.google.com/']

service = Create_Service(CLIENT_SECRET_FILE, API_NAME, API_VERSION, SCOPES)



client_secrets.json-gmail-v1-(['https://mail.google.com/'],)
['https://mail.google.com/']
Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=571222025068-m9ovg7f56g0ldejvv4j7it2j8k3i4ulo.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fmail.google.com%2F&state=XQzrpJJ2JSTOHCyeZTUtECclokekce&access_type=offline
gmail service created successfully
{'id': '17d22f7a149f19f6', 'threadId': '17d22f7a149f19f6', 'labelIds': ['SENT']}


# Write Email Function

In [3]:
def write_email(send_to_email, message):
    emailMsg = message
    mimeMessage = MIMEMultipart()
    mimeMessage['to'] = send_to_email
    mimeMessage['subject'] = 'Python DeCal Admission'
    mimeMessage.attach(MIMEText(emailMsg, 'plain'))
    raw_string = base64.urlsafe_b64encode(mimeMessage.as_bytes()).decode()

    message = service.users().messages().send(userId='me', body={'raw': raw_string}).execute()
    print(message)

# Read in Preconfigured Email Data

In [4]:
import pandas as pd

In [7]:
email_data = pd.read_csv("email_list.csv")

email_data

,Unnamed: 0,Emails,Messages
0,0,jamessunseri@berkeley.edu,"Hello James Sunseri,\n\n We look forwar..."


In [8]:
print(email_data['Emails'][0], email_data['Messages'][0])

jamessunseri@berkeley.edu     Hello James Sunseri,

    We look forward to having you in our class this fall! Here is the permission code for the decal: 53363

    Please let me know if you have any questions

    Onwards,
    Python Decal Staff


In [9]:
write_email(email_data['Emails'][0], email_data['Messages'][0])

{'id': '17d230dcfc4881cb', 'threadId': '17d230dcfc4881cb', 'labelIds': ['UNREAD', 'SENT', 'INBOX']}
